## Lab 1.1 - Analyzing Real Estate Data in St. Petersburg
We have data from Yandex.Realty classified https://realty.yandex.ru containing real estate listings for apartments in St. Petersburg and Leningrad Oblast from 2016 till the middle of August 2018. In this Lab you'll learn how to work with datasets in python using pandas library for data analysis using the provided data. 
For datasets like this it becomes hard to use Excel because of its size. If you learn how to work with pandas you'll be able to do basic analytical research much faster than in Excel.
To execute code IPython notebook click on the Cell and press Shift-Enter.
If you don't have any experience with Jupyter notebooks, please look at this tutorial: https://mybinder.org/v2/gh/ipython/ipython-in-depth/master?filepath=binder/Index.ipynb

### Main objectives
After successful completion of the lab work students will be able to:
-	Analyze large datasets that don't work well in Excel using IPython notebook and pandas library
-   Find outliers and remove them in the datasets
-	Calculate different statistics on data fields


### Tasks
-	Calculate median and mean sell and rent prices in St. Petersburg
-   Find outliers: too cheap or too expensive apartments which seem to be a wrong data. Remove them from the dataset.
-	Find houses with the most cheapest and most expensive prices per square meter.
-   Analyze how many rent offers have the commission and what is the most popular commission



In [1]:
# let's import pandas library and set options to be able to view data right in the browser
# Assign pd for a short alias for pandas library
import pandas as pd
# Call set_option function fron pandas library to edit them limit for number of the displayed
# columns, width of all columns and maximum width of one column
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', 1000)

In [2]:
# let's look what is inside our data file to understand how we can export it. 
# We can use linux command "head" which shows the first row in the file. 
# In IPython notebooks you can execute linux commands using "!" prefixe before the command name. 
# This mechanism is called "IPython magic"
!head -n 3 data/spb.real.estate.archive.2018.tsv

offer_id	first_day_exposition	last_day_exposition	last_price	floor	open_plan	rooms	studio	area	kitchen_area	living_area	agent_fee	renovation	offer_type	category_type	unified_address	building_id
5490504238877583424	2014-11-27T00:00:00+03:00	2016-03-14T00:00:00+03:00	3676740.0	19	False	1	False	27.4	16.56	16.56		0.0	1	2	Россия, Санкт-Петербург, улица Шкапина, 9-11	3552413344225333068
5490504238903444021	2014-11-27T00:00:00+03:00	2016-04-28T00:00:00+03:00	7038880.0	2	False	3	False	85.84	19.64	41.2		0.0	1	2	Россия, Санкт-Петербург, Кушелевская дорога, 3к4	7274736308217971906


In [3]:
# if you don't remember what linux command does (i.e. what is -n), you can always look for help
!head --help

Usage: head [OPTION]... [FILE]...
Print the first 10 lines of each FILE to standard output.
With more than one FILE, precede each with a header giving the file name.

With no FILE, or when FILE is -, read standard input.

Mandatory arguments to long options are mandatory for short options too.
  -c, --bytes=[-]NUM       print the first NUM bytes of each file;
                             with the leading '-', print all but the last
                             NUM bytes of each file
  -n, --lines=[-]NUM       print the first NUM lines instead of the first 10;
                             with the leading '-', print all but the last
                             NUM lines of each file
  -q, --quiet, --silent    never print headers giving file names
  -v, --verbose            always print headers giving file names
  -z, --zero-terminated    line delimiter is NUL, not newline
      --help     display this help and exit
      --version  output version information and exit

NUM may have a mu

In [4]:
!ls

 Building_models.ipynb		     ml_model_tips.ipynb
 catboost_info			     model.pkl
 cleaned_dataset.csv		     scaler_x.pkl
 cleaned_dataset_my.csv		     scaler_y.pkl
 data				    'spb.real.estate.archive.2018 (1).tsv'
 lab1_1_EDA_real_estate_data.ipynb   spb.real.estate.archive.2018.tsv
 lab1_2_visualization.ipynb	     spb.real.estate.archive.sample5000.tsv
 lab2_building_model-Copy1.ipynb     Untitled.ipynb
 lab2_building_model.ipynb	     visual_tips.ipynb
'ml_model_tips (1).ipynb'


In [5]:
# we see that it's tab separated file, we can load it with pd.read_table method
# load our dataset and see which data it contains.
# HINT: you can type 'Tab' after starting typing the filename and IPython will automatically fill it for you
# After this command a new variable will be created (spb_df) containing pandas dataframe with data in our file
# You can think of dataframe structure as an analogue of Excel sheet
spb_df = pd.read_table('data/spb.real.estate.archive.2018.tsv')

In [6]:
# HINT: if you don't know which arguments any function takes or what it does you can check the documenation right 
# in the notebook just by adding "?" after the function name. Let's look at pandas read_table function:
pd.read_table?

Signature:
pd.read_table(
    filepath_or_buffer: 'FilePath | ReadCsvBuffer[bytes] | ReadCsvBuffer[str]',
    *,
    sep: 'str | None | lib.NoDefault' = <no_default>,
    delimiter: 'str | None | lib.NoDefault' = None,
    header: "int | Sequence[int] | None | Literal['infer']" = 'infer',
    names: 'Sequence[Hashable] | None | lib.NoDefault' = <no_default>,
    index_col: 'IndexLabel | Literal[False] | None' = None,
    usecols=None,
    squeeze: 'bool | None' = None,
    prefix: 'str | lib.NoDefault' = <no_default>,
    mangle_dupe_cols: 'bool' = True,
    dtype: 'DtypeArg | None' = None,
    engine: 'CSVEngine | None' = None,
    converters=None,
    true_values=None,
    false_values=None,
    skipinitialspace: 'bool' = False,
    skiprows=None,
    skipfooter: 'int' = 0,
    nrows: 'int | None' = None,
    na_values=None,
    keep_default_na: 'bool' = True,
    na_filter: 'bool' = True,
    verbose: 'bool' = False,
    skip_blank_lines: 'bool' = True,
    parse_dates=False,
    in

In [8]:
# let's check how much data in terms of rows we have where each row represent 
# one offer from Yandex.Realty
len(spb_df)

429187

In [16]:
spb_df.describe()

,offer_id,last_price,floor,rooms,area,kitchen_area,living_area,agent_fee,renovation,offer_type,category_type,building_id
count,4.291870e+05,4.291870e+05,429187.000000,429187.000000,429187.000000,341591.000000,364544.000000,149229.000000,295659.000000,429187.000000,429187.0,4.291870e+05
mean,4.840885e+18,4.202162e+06,6.673296,1.778169,57.423093,11.526046,32.627916,65.705497,3.677206,1.398861,2.0,4.580242e+18
std,2.712898e+18,9.527811e+06,5.248549,1.060154,33.121345,48.717769,20.880149,33.820077,4.187101,0.489665,0.0,2.717262e+18
min,7.797270e+05,1.300000e+02,1.000000,0.000000,6.000000,1.040000,1.500000,0.000000,0.000000,1.000000,2.0,4.585730e+14
25%,2.573081e+18,2.800000e+04,3.000000,1.000000,37.500000,8.000000,18.000000,50.000000,0.000000,1.000000,2.0,2.153540e+18
50%,5.153034e+18,2.850000e+06,5.000000,2.000000,48.600000,10.000000,29.000000,60.000000,3.000000,1.000000,2.0,4.652023e+18
75%,7.140224e+18,5.170000e+06,9.000000,2.000000,67.000000,12.980000,40.000000,100.000000,7.000000,2.000000,2.0,6.919866e+18
max,9.223371e+18,7.630000e+08,92.000000,22.000000,1000.000000,25000.000000,2015.000000,100.000000,14.000000,2.000000,2.0,9.220758e+18


In [8]:
# let's look at random sample of size 10 of the loaded dataset to understand what's inside
spb_df.sample(10)

,offer_id,first_day_exposition,last_day_exposition,last_price,floor,open_plan,rooms,studio,area,kitchen_area,living_area,agent_fee,renovation,offer_type,category_type,unified_address,building_id
228994,8295435159644179200,2017-10-12T00:00:00+03:00,2017-10-30T00:00:00+03:00,25000.0,8,False,2,False,66.0,12.0,36.0,50.0,11.0,2,2,"Россия, Санкт-Петербург, улица Костюшко, 2",4748912576298422795
77768,2575825030533508609,2014-11-27T00:00:00+03:00,2016-09-18T00:00:00+03:00,5500000.0,4,False,3,False,58.0,6.0,39.0,NaN,3.0,1,2,"Россия, Санкт-Петербург, улица Есенина, 36к1",7215469126417717631
375891,5565310413830692591,2018-06-18T00:00:00+03:00,2018-06-25T00:00:00+03:00,32000.0,1,False,0,True,38.0,NaN,NaN,100.0,NaN,2,2,"Россия, Санкт-Петербург, Пушкин, Анциферовская улица, 9В",21512832639383788
111548,2554639205084019543,2016-11-28T00:00:00+03:00,2018-08-20T00:00:00+03:00,2800000.0,13,False,1,False,39.2,8.6,18.5,NaN,0.0,1,2,"Россия, Ленинградская область, Всеволожский район, деревня Новое Девяткино, улица Энергетиков, 1",8427373449207267761
7412,9069199666260049469,2015-07-08T00:00:00+03:00,2018-03-11T00:00:00+03:00,8900000.0,2,False,3,False,104.0,9.0,77.4,NaN,NaN,1,2,"Россия, Санкт-Петербург, Гороховая улица, 32",70279605270879388
313394,7941511182225100288,2018-03-22T00:00:00+03:00,2018-07-16T00:00:00+03:00,2350000.0,2,False,2,False,47.8,5.0,35.0,NaN,NaN,1,2,"Россия, Ленинградская область, Приозерский район, Мичуринское сельское поселение, поселок Мичуринское, Озерный переулок, 1",8768765023262985403
187838,7092019215919506688,2017-06-23T00:00:00+03:00,2017-08-23T00:00:00+03:00,19000.0,1,False,1,False,35.0,10.0,15.0,60.0,3.0,2,2,"Россия, Санкт-Петербург, Светлановский проспект, 74к1",2852157866563500542
405678,7569506429740795760,2018-02-01T00:00:00+03:00,2018-08-16T00:00:00+03:00,16000.0,3,False,1,False,31.0,6.0,18.0,50.0,11.0,2,2,"Россия, Санкт-Петербург, Красное Село, улица Лермонтова, 14",8763959950889276838
184933,1855460378608114688,2017-06-14T00:00:00+03:00,2018-03-21T00:00:00+03:00,10600000.0,8,False,2,False,65.0,13.0,35.0,NaN,2.0,1,2,"Россия, Санкт-Петербург, Республиканская улица, 24к1",3395537759275353752
412151,5175901685917225294,2016-12-28T00:00:00+03:00,2018-08-07T00:00:00+03:00,36000.0,12,False,2,False,70.0,18.0,42.0,100.0,NaN,2,2,"Россия, Санкт-Петербург, улица Матроса Железняка, 57",2564689027470597295


In [9]:
len(spb_df)

429187

In [10]:
spb_df.sample(5000).to_csv('spb.real.estate.archive.sample5000.tsv', index = False, sep='\t')

In [11]:
!pwd

/home/jovyan/ML


### Analyze prices for rent and sell
We see that we have both RENT and SELL data. 
<p>offer_type column contains data to distinct them, 2 stands for RENT, 1 for SELL


#### Analyze rent prices

In [12]:
# at first let's split our dataframe to rent and sell
# Select some part of dataframe according to conditions in brackets
rent_df = spb_df[spb_df.offer_type == 2]
sell_df = spb_df[spb_df.offer_type == 1]
# .format function allows you to put variales' values instead of {} in the final string
print("Total rent data size: {}".format(len(rent_df)))
print("Total sell data size: {}".format(len(sell_df)))

Total rent data size: 171186
Total sell data size: 258001


In [13]:
# let's check median and price for renting apartments
# .last_price indicates that you use data just from the column named last_price
rent_median_price = rent_df.last_price.median()
rent_mean_price = rent_df.last_price.mean()
print("Rent median price: {}".format(rent_median_price))
print("Rent mean price: {}".format(rent_mean_price))

Rent median price: 25000.0
Rent mean price: 31719.256638977487


In [14]:
# Just to look at data of one column you can select this column. 
# 'head' function will take only first rows 
rent_df.last_price.head(10)

1086    20000.0
1096    40000.0
1097    45000.0
1149    21000.0
1514    25000.0
1515    35000.0
1516    22000.0
1537    20000.0
1555    26000.0
1695    25000.0
Name: last_price, dtype: float64

#### Let's look at the most expensive apartments in rent

In [15]:
# .max function returns the maximum value across all rows
max_price_rent = rent_df.last_price.max()

In [16]:
print("Max price in rent: {}".format(max_price_rent))

Max price in rent: 4900000.0


In [17]:
# Now select all offers where price is equal to maximum value across all rent prices
rent_df_max = rent_df[rent_df.last_price == max_price_rent]

In [18]:
rent_df_max

,offer_id,first_day_exposition,last_day_exposition,last_price,floor,open_plan,rooms,studio,area,kitchen_area,living_area,agent_fee,renovation,offer_type,category_type,unified_address,building_id
123675,5594900220237251840,2017-01-16T00:00:00+03:00,2017-01-16T00:00:00+03:00,4900000.0,3,False,3,False,55.4,5.5,42.1,NaN,0.0,2,2,"Россия, Санкт-Петербург, Новочеркасский проспект, 36",7844665691371923607


We can see that it was probably a mistake. It's hard to imagine why an aparment could cost 4.9 mln rubles per month. To analyze it let's look at the random sample of rent offers with prices of more than 1 mln rubles.

In [19]:
# Knowing real estate market in St.Petersburg we understand 
# that offers with rent prices over 1 million rubles are outliers
outliers_rent = rent_df[rent_df.last_price > 1000000]
print("Number of outliers in rent: {}".format(len(outliers_rent)))

Number of outliers in rent: 22


In [20]:
outliers_rent.sample(5)

,offer_id,first_day_exposition,last_day_exposition,last_price,floor,open_plan,rooms,studio,area,kitchen_area,living_area,agent_fee,renovation,offer_type,category_type,unified_address,building_id
12990,3988133,2015-09-19T00:00:00+03:00,2016-04-13T00:00:00+03:00,3450000.0,19,False,1,True,30.0,NaN,NaN,NaN,3.0,2,2,"Россия, Санкт-Петербург, посёлок Парголово, Заречная улица, 19к1",2079266574874175179
326408,8492010403417061889,2018-04-06T00:00:00+03:00,2018-04-24T00:00:00+03:00,3500000.0,8,False,0,True,30.0,NaN,NaN,NaN,NaN,2,2,"Россия, Санкт-Петербург, Репищева улица, 10к1",1967793054474181316
397740,8535940262361472000,2018-07-12T00:00:00+03:00,2018-07-16T00:00:00+03:00,2900000.0,2,False,1,False,29.3,7.2,15.1,NaN,NaN,2,2,"Россия, Санкт-Петербург, Альпийский переулок, 22",2164916565767642422
149082,8260656212628561921,2017-03-14T00:00:00+03:00,2017-05-18T00:00:00+03:00,4300000.0,4,False,1,False,38.0,9.0,18.0,NaN,0.0,2,2,"Россия, Санкт-Петербург, Пушкин, Оранжерейная улица, 39",1604138570197506217
29410,4944609,2015-12-23T00:00:00+03:00,2016-05-10T00:00:00+03:00,2250000.0,4,False,1,False,35.0,9.0,17.0,100.0,3.0,2,2,"Россия, Ленинградская область, Тосно, проспект Ленина, 27",4242058865805744796


In [21]:
# We see that all of these offers are results of user mistakes.
# Pause to think what kind of mistakes might cause such numbers?
# Let's now look at rent offers below 1M but more than 500K

In [22]:
rent_df[(rent_df.last_price > 500000) & (rent_df.last_price < 1000000)]

,offer_id,first_day_exposition,last_day_exposition,last_price,floor,open_plan,rooms,studio,area,kitchen_area,living_area,agent_fee,renovation,offer_type,category_type,unified_address,building_id
83621,6302977522903470578,2015-05-28T00:00:00+03:00,2017-04-26T00:00:00+03:00,550000.0,6,False,4,False,180.0,18.0,122.0,NaN,0.0,2,2,"Россия, Санкт-Петербург, Парадная улица, 3к1",7861463534572863507
89836,6302977522901450489,2016-09-14T00:00:00+03:00,2016-09-15T00:00:00+03:00,800000.0,2,False,2,False,80.0,20.0,40.0,NaN,0.0,2,2,"Россия, Санкт-Петербург, Большая Морская улица, 27",8010764606439654926
106222,4377907628678297857,2016-11-07T00:00:00+03:00,2017-03-13T00:00:00+03:00,600000.0,6,False,4,False,187.0,NaN,127.0,100.0,1.0,2,2,"Россия, Санкт-Петербург, Парадная улица, 3к1",7861463534572863507
172166,3376106655979657514,2017-02-24T00:00:00+03:00,2017-05-24T00:00:00+03:00,600000.0,6,False,4,False,200.0,NaN,103.0,100.0,0.0,2,2,"Россия, Санкт-Петербург, Парадная улица, 3к2",7861463534572863508
172950,1896730477252952293,2016-10-31T00:00:00+03:00,2017-05-26T00:00:00+03:00,650000.0,6,False,4,False,190.0,20.0,102.0,100.0,0.0,2,2,"Россия, Санкт-Петербург, Парадная улица, 3к1",7861463534572863507
253075,9174606537144832259,2017-11-16T00:00:00+03:00,2018-08-20T00:00:00+03:00,600000.0,1,False,4,False,165.0,NaN,115.0,NaN,6.0,2,2,"Россия, Санкт-Петербург, проспект Динамо, 6",135575281136523355
321216,2596865999798823681,2018-03-31T00:00:00+03:00,2018-04-04T00:00:00+03:00,575000.0,3,False,4,False,120.0,NaN,NaN,50.0,1.0,2,2,"Россия, Санкт-Петербург, Кременчугская улица, 17к3",9078299753075959719
323711,9106659135022232833,2018-04-03T00:00:00+03:00,2018-06-15T00:00:00+03:00,600000.0,1,False,4,False,90.0,13.0,54.0,0.0,11.0,2,2,"Россия, Санкт-Петербург, Большой Сампсониевский проспект, 33/1",192308642065385452
336616,8867625833179319559,2018-04-21T00:00:00+03:00,2018-08-09T00:00:00+03:00,600000.0,1,False,4,False,117.0,20.0,83.0,0.0,NaN,2,2,"Россия, Санкт-Петербург, проспект Динамо, 6",135575281136523355
419166,5152264856292210972,2018-02-01T00:00:00+03:00,2018-08-16T00:00:00+03:00,600000.0,1,False,4,False,140.0,95.0,100.0,100.0,NaN,2,2,"Россия, Санкт-Петербург, проспект Динамо, 6",135575281136523355


In [23]:
# for these cases sometimes it seems that the price is correct - in the unified_address column we
# see that these offers are from elite houses
# like here: https://realty.yandex.ru/archive/Россия%2C%20Санкт-Петербург%2C%20проспект%20Динамо%2C%206/snyat/kvartira/
# but for some of them price seems to be 10 times more than it should be, like here
# https://realty.yandex.ru/archive/Россия%2C%20Санкт-Петербург%2C%20Большая%20Морская%20улица%2C%2027/snyat/kvartira/
# let's find these cases

### Calculate price per square meter, get median prices for house and find outliers with the help of this

In [24]:
# Create a new column price_per_sq_m and calculate price per sq m
rent_df['price_per_sq_m'] = rent_df.last_price/rent_df.area

/tmp/ipykernel_6846/816257644.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rent_df['price_per_sq_m'] = rent_df.last_price/rent_df.area


In [25]:
rent_df.head(5)

,offer_id,first_day_exposition,last_day_exposition,last_price,floor,open_plan,rooms,studio,area,kitchen_area,living_area,agent_fee,renovation,offer_type,category_type,unified_address,building_id,price_per_sq_m
1086,1100202,2015-01-24T00:00:00+03:00,2016-01-19T00:00:00+03:00,20000.0,7,False,1,False,28.0,8.0,20.0,100.0,3.0,2,2,"Россия, Санкт-Петербург, Богатырский проспект, 5к1",6719200042030752141,714.285714
1096,5028522699704970725,2015-01-25T00:00:00+03:00,2016-01-20T00:00:00+03:00,40000.0,4,False,2,False,72.0,10.0,44.0,NaN,0.0,2,2,"Россия, Санкт-Петербург, набережная реки Фонтанки, 66",3406104650917898355,555.555556
1097,5028522699709767236,2015-01-25T00:00:00+03:00,2016-01-21T00:00:00+03:00,45000.0,2,False,2,False,55.0,NaN,33.0,NaN,0.0,2,2,"Россия, Санкт-Петербург, Большой проспект П.С., 102",2357095685903407888,818.181818
1149,1100874,2015-01-28T00:00:00+03:00,2016-01-16T00:00:00+03:00,21000.0,5,False,1,False,35.0,8.0,18.0,NaN,11.0,2,2,"Россия, Санкт-Петербург, Лужская улица, 8",5887402119500981764,600.000000
1514,875413,2015-02-13T00:00:00+03:00,2016-01-23T00:00:00+03:00,25000.0,10,False,1,True,44.0,15.0,18.0,90.0,10.0,2,2,"Россия, Санкт-Петербург, Учительская улица, 3",6527910144166454997,568.181818


In [26]:
# find what's median and mean price per square meter in rent
median_price_per_sq_m = rent_df.price_per_sq_m.median()
mean_price_per_sq_m = rent_df.price_per_sq_m.mean()
print("Median price per sq m in rent: {}".format(median_price_per_sq_m))
print("Mean price per sq m in rent: {}".format(mean_price_per_sq_m))

Median price per sq m in rent: 550.0
Mean price per sq m in rent: 600.1110692967684


#### Find median price per sq m per house

In [27]:
# .groupby function groups dataframe by a selected column (unified_address)
# For each address we then take a median of price per sq m and return a new dataframe 
# with a result
house_rent_df = rent_df.groupby('unified_address').price_per_sq_m.median().reset_index()

In [28]:
house_rent_df.head(5)

,unified_address,price_per_sq_m
0,"Россия, Ленинградская область, Бокситогорск, Комсомольская улица, 5А",171.428571
1,"Россия, Ленинградская область, Бокситогорск, Южная улица, 15",250.000000
2,"Россия, Ленинградская область, Бокситогорск, улица Металлургов, 7",164.948454
3,"Россия, Ленинградская область, Бокситогорск, улица Павлова, 37",138.888889
4,"Россия, Ленинградская область, Бокситогорский район, Пикалёво, 6-й микрорайон, 25",195.652174


In [29]:
# Rename the column with a 'rename' function in this new dataframe
house_rent_df.rename(columns = {'price_per_sq_m': 'house_price_sqm_median'}, inplace = True)

#### Merge rent data with house median prices and inspect outliers

In [30]:
# With a 'merge' function join two dataframes.
# As these 2 dataframes have only one common column name (unified_address) it will merge dataframes by it
rent_df = rent_df.merge(house_rent_df)

In [31]:
# Assign outliers if price per sq m is 5 times higher than the median value in the house
outliers = rent_df[(rent_df.price_per_sq_m/rent_df.house_price_sqm_median) > 5]

In [32]:
print(len(outliers))

49


In [33]:
outliers.sample(10)

,offer_id,first_day_exposition,last_day_exposition,last_price,floor,open_plan,rooms,studio,area,kitchen_area,living_area,agent_fee,renovation,offer_type,category_type,unified_address,building_id,price_per_sq_m,house_price_sqm_median
74350,7710633944129267457,2017-08-07T00:00:00+03:00,2017-11-07T00:00:00+03:00,4500000.0,7,False,1,False,63.1,16.1,23.4,NaN,0.0,2,2,"Россия, Санкт-Петербург, Колпино, улица Ижорского Батальона, 18",6064043689074707027,71315.372425,397.674419
132282,7521881594612635746,2018-05-14T00:00:00+03:00,2018-05-21T00:00:00+03:00,4700000.0,10,False,1,False,40.3,8.3,20.0,100.0,NaN,2,2,"Россия, Санкт-Петербург, Ленинский проспект, 111к2",2371480959169104773,116625.310174,602.631579
1992,3988133,2015-09-19T00:00:00+03:00,2016-04-13T00:00:00+03:00,3450000.0,19,False,1,True,30.0,NaN,NaN,NaN,3.0,2,2,"Россия, Санкт-Петербург, посёлок Парголово, Заречная улица, 19к1",2079266574874175179,115000.000000,571.428571
14032,3277920237451437825,2016-05-10T00:00:00+03:00,2016-05-14T00:00:00+03:00,200000.0,12,False,1,False,45.0,9.0,20.0,50.0,3.0,2,2,"Россия, Санкт-Петербург, посёлок Парголово, улица Михаила Дудина, 25к1",4661483057276827065,4444.444444,475.000000
143498,7796157803399736832,2018-06-12T00:00:00+03:00,2018-07-12T00:00:00+03:00,150000.0,11,False,1,False,42.0,NaN,NaN,NaN,NaN,2,2,"Россия, Санкт-Петербург, Пулковское шоссе, 38к7",6690775821291644660,3571.428571,641.025641
120554,9106659135022232833,2018-04-03T00:00:00+03:00,2018-06-15T00:00:00+03:00,600000.0,1,False,4,False,90.0,13.0,54.0,0.0,11.0,2,2,"Россия, Санкт-Петербург, Большой Сампсониевский проспект, 33/1",192308642065385452,6666.666667,888.888889
156301,8535940262361472000,2018-07-12T00:00:00+03:00,2018-07-16T00:00:00+03:00,2900000.0,2,False,1,False,29.3,7.2,15.1,NaN,NaN,2,2,"Россия, Санкт-Петербург, Альпийский переулок, 22",2164916565767642422,98976.109215,576.707727
34325,4266909851082118912,2017-01-14T00:00:00+03:00,2017-05-27T00:00:00+03:00,3100000.0,2,False,1,False,39.0,7.0,20.0,NaN,3.0,2,2,"Россия, Санкт-Петербург, Петергоф, Озерковая улица, 39к1",6053955221850724766,79487.179487,466.666667
98844,2877221490003907329,2018-01-10T00:00:00+03:00,2018-02-09T00:00:00+03:00,200000.0,5,False,1,False,40.0,NaN,NaN,NaN,NaN,2,2,"Россия, Санкт-Петербург, улица Есенина, 14к2",7215469126417656128,5000.000000,606.060606
22530,4281053917652545793,2016-09-02T00:00:00+03:00,2016-09-05T00:00:00+03:00,200000.0,2,False,1,False,35.0,9.0,18.0,100.0,0.0,2,2,"Россия, Санкт-Петербург, Пушкин, Средняя улица, 13",536301776913345184,5714.285714,536.111111


#### Clean data from the outliers

In [34]:
# Let's create a new dataframe with clean data.
# With ~ we can select the data which does NOT correspond to the required conditions
rent_df_cleaned = rent_df[~((rent_df.price_per_sq_m/rent_df.house_price_sqm_median) > 5)]

In [35]:
# check that in the middle segment everything is fine now
rent_df_cleaned[(rent_df_cleaned.last_price > 500000) & (rent_df_cleaned.last_price < 1000000)]

,offer_id,first_day_exposition,last_day_exposition,last_price,floor,open_plan,rooms,studio,area,kitchen_area,living_area,agent_fee,renovation,offer_type,category_type,unified_address,building_id,price_per_sq_m,house_price_sqm_median
21420,6302977522903470578,2015-05-28T00:00:00+03:00,2017-04-26T00:00:00+03:00,550000.0,6,False,4,False,180.0,18.0,122.0,NaN,0.0,2,2,"Россия, Санкт-Петербург, Парадная улица, 3к1",7861463534572863507,3055.555556,1442.307692
27223,4377907628678297857,2016-11-07T00:00:00+03:00,2017-03-13T00:00:00+03:00,600000.0,6,False,4,False,187.0,NaN,127.0,100.0,1.0,2,2,"Россия, Санкт-Петербург, Парадная улица, 3к1",7861463534572863507,3208.556150,1442.307692
57890,3376106655979657514,2017-02-24T00:00:00+03:00,2017-05-24T00:00:00+03:00,600000.0,6,False,4,False,200.0,NaN,103.0,100.0,0.0,2,2,"Россия, Санкт-Петербург, Парадная улица, 3к2",7861463534572863508,3000.000000,1385.501355
58474,1896730477252952293,2016-10-31T00:00:00+03:00,2017-05-26T00:00:00+03:00,650000.0,6,False,4,False,190.0,20.0,102.0,100.0,0.0,2,2,"Россия, Санкт-Петербург, Парадная улица, 3к1",7861463534572863507,3421.052632,1442.307692
91716,9174606537144832259,2017-11-16T00:00:00+03:00,2018-08-20T00:00:00+03:00,600000.0,1,False,4,False,165.0,NaN,115.0,NaN,6.0,2,2,"Россия, Санкт-Петербург, проспект Динамо, 6",135575281136523355,3636.363636,3636.363636
125902,8867625833179319559,2018-04-21T00:00:00+03:00,2018-08-09T00:00:00+03:00,600000.0,1,False,4,False,117.0,20.0,83.0,0.0,NaN,2,2,"Россия, Санкт-Петербург, проспект Динамо, 6",135575281136523355,5128.205128,3636.363636
166293,5152264856292210972,2018-02-01T00:00:00+03:00,2018-08-16T00:00:00+03:00,600000.0,1,False,4,False,140.0,95.0,100.0,100.0,NaN,2,2,"Россия, Санкт-Петербург, проспект Динамо, 6",135575281136523355,4285.714286,3636.363636


In [36]:
# look at offers with price > 1 000 000
rent_df_cleaned[(rent_df_cleaned.last_price >= 1000000)]

,offer_id,first_day_exposition,last_day_exposition,last_price,floor,open_plan,rooms,studio,area,kitchen_area,living_area,agent_fee,renovation,offer_type,category_type,unified_address,building_id,price_per_sq_m,house_price_sqm_median
5900,5199154,2016-01-25T00:00:00+03:00,2016-01-26T00:00:00+03:00,3800000.0,3,False,2,False,51.0,6.0,30.5,NaN,0.0,2,2,"Россия, Санкт-Петербург, Пискарёвский проспект, 149",7757194067653498771,74509.803922,74509.803922
21528,4075481786711135488,2016-08-18T00:00:00+03:00,2016-08-18T00:00:00+03:00,3500000.0,1,False,2,False,55.0,8.3,30.5,NaN,3.0,2,2,"Россия, Ленинградская область, Всеволожск, микрорайон Южный, Центральная улица, 13",4521154714631889065,63636.363636,63636.363636
25525,8743101202301731585,2016-10-13T00:00:00+03:00,2016-11-12T00:00:00+03:00,1500000.0,3,False,2,False,54.7,NaN,NaN,NaN,7.0,2,2,"Россия, Ленинградская область, Выборгский район, Светогорское городское поселение, поселок городского типа Лесогорский, улица Труда, 7",2755098623625145659,27422.303473,13811.700183
105860,4088017598264857345,2018-02-09T00:00:00+03:00,2018-03-21T00:00:00+03:00,1330000.0,3,False,1,False,33.0,6.0,19.0,NaN,3.0,2,2,"Россия, Ленинградская область, Луга, проспект Урицкого, 65",1792010144615909485,40303.030303,40303.030303


In [37]:
# we see, that all of them are outliers, they were not cleaned by the house price per sq m median 
# because it was the only one or just one of two offers per house
# let's clean them also
# Store result in the same dataframe
rent_df_cleaned = rent_df_cleaned[rent_df_cleaned.last_price < 1000000]

In [38]:
# let's examine apartments with big price per sq m
rent_df_cleaned[rent_df_cleaned.price_per_sq_m > 3000]

,offer_id,first_day_exposition,last_day_exposition,last_price,floor,open_plan,rooms,studio,area,kitchen_area,living_area,agent_fee,renovation,offer_type,category_type,unified_address,building_id,price_per_sq_m,house_price_sqm_median
14921,7102677476104578816,2016-05-20T00:00:00+03:00,2016-05-20T00:00:00+03:00,270000.0,1,False,1,False,80.0,NaN,NaN,0.0,3.0,2,2,"Россия, Санкт-Петербург, Загородный проспект, 5",8810747501928407960,3375.000000,1978.197674
17068,159653575951184384,2016-06-16T00:00:00+03:00,2016-07-01T00:00:00+03:00,300000.0,2,False,3,False,54.0,8.0,35.0,50.0,11.0,2,2,"Россия, Санкт-Петербург, 18-я линия В.О., 43",6003296125598219505,5555.555556,5555.555556
21420,6302977522903470578,2015-05-28T00:00:00+03:00,2017-04-26T00:00:00+03:00,550000.0,6,False,4,False,180.0,18.0,122.0,NaN,0.0,2,2,"Россия, Санкт-Петербург, Парадная улица, 3к1",7861463534572863507,3055.555556,1442.307692
27223,4377907628678297857,2016-11-07T00:00:00+03:00,2017-03-13T00:00:00+03:00,600000.0,6,False,4,False,187.0,NaN,127.0,100.0,1.0,2,2,"Россия, Санкт-Петербург, Парадная улица, 3к1",7861463534572863507,3208.556150,1442.307692
49079,3793550987719225600,2017-03-19T00:00:00+03:00,2017-11-02T00:00:00+03:00,310000.0,3,False,4,False,100.0,14.0,71.0,100.0,1.0,2,2,"Россия, Санкт-Петербург, Лиговский проспект, 123",2861706125080798064,3100.000000,976.307190
58474,1896730477252952293,2016-10-31T00:00:00+03:00,2017-05-26T00:00:00+03:00,650000.0,6,False,4,False,190.0,20.0,102.0,100.0,0.0,2,2,"Россия, Санкт-Петербург, Парадная улица, 3к1",7861463534572863507,3421.052632,1442.307692
61671,1896730477284436579,2017-05-31T00:00:00+03:00,2017-05-31T00:00:00+03:00,230000.0,9,False,2,False,67.0,14.0,30.0,100.0,0.0,2,2,"Россия, Санкт-Петербург, Глухая Зеленина улица, 6",6445545084936583797,3432.835821,923.076923
64008,1492050389917501301,2017-06-07T00:00:00+03:00,2017-06-07T00:00:00+03:00,195000.0,4,False,1,False,57.0,8.0,19.0,100.0,0.0,2,2,"Россия, Санкт-Петербург, проспект Добролюбова, 2",6684242979485775735,3421.052632,779.220779
64019,1492050390917641037,2016-10-28T00:00:00+03:00,2017-06-07T00:00:00+03:00,400000.0,4,False,3,False,130.0,13.0,85.0,100.0,0.0,2,2,"Россия, Санкт-Петербург, Константиновский проспект, 23",5091810811209082905,3076.923077,3484.848485
64177,1492050389884015440,2017-06-07T00:00:00+03:00,2017-06-08T00:00:00+03:00,460000.0,4,False,3,False,130.0,13.0,80.0,100.0,0.0,2,2,"Россия, Санкт-Петербург, Константиновский проспект, 23",5091810811209082905,3538.461538,3484.848485


In [39]:
# we can see that outliers are those which are in houses with price per sq m below 1000 rubles, 
# and those which are the same with median, in which case it's the only offer per house
outliers_count = len(rent_df_cleaned[(rent_df_cleaned.price_per_sq_m > 3000) 
                                     & ((rent_df_cleaned.house_price_sqm_median < 1000) 
                                        | (rent_df_cleaned.house_price_sqm_median == rent_df_cleaned.price_per_sq_m))])
print("outliers found: {}".format(outliers_count))
rent_df_cleaned = rent_df_cleaned[~((rent_df_cleaned.price_per_sq_m > 3000) 
                                     & ((rent_df_cleaned.house_price_sqm_median < 1000) 
                                        | (rent_df_cleaned.house_price_sqm_median == rent_df_cleaned.price_per_sq_m)))]

outliers found: 8


In [40]:
# now max price should be correct, lets look at this offer
max_rent_apartment = rent_df_cleaned[rent_df_cleaned.last_price == rent_df_cleaned.last_price.max()]
max_rent_apartment

,offer_id,first_day_exposition,last_day_exposition,last_price,floor,open_plan,rooms,studio,area,kitchen_area,living_area,agent_fee,renovation,offer_type,category_type,unified_address,building_id,price_per_sq_m,house_price_sqm_median
58474,1896730477252952293,2016-10-31T00:00:00+03:00,2017-05-26T00:00:00+03:00,650000.0,6,False,4,False,190.0,20.0,102.0,100.0,0.0,2,2,"Россия, Санкт-Петербург, Парадная улица, 3к1",7861463534572863507,3421.052632,1442.307692


In [41]:
# let's look at house address. 
# with 'iloc' property we can take any row by index, 
# in this case we take the first item to get the address
max_rent_apartment.unified_address.iloc[0]

'Россия, Санкт-Петербург, Парадная улица, 3к1'

In [42]:
# impoort urllib to prepare correct URLs for looking at offers archive 
# at https://realty.yandex.ru/archive
import urllib

In [43]:
# if you're interested in images of this apartment you can find it here: 
# https://realty.yandex.ru/archive/Россия%2C%20Санкт-Петербург%2C%20Парадная%20улица%2C%203к1/snyat/kvartira/2/?roomsTotal=4

# let's print the URL of the page, on which we can find our offer
# we use urllib.parse.quote function to get URL part from our address, otherwise URL wouldn't be correct
# print("https://realty.yandex.ru/archive/{}/snyat/kvartira/2/?roomsTotal=4".format(urllib.parse.quote(max_rent_apartment.unified_address.iloc[0])))
# print("https://realty.yandex.ru/otsenka-kvartiry-po-adresu-onlayn/{}/snyat/kvartira/?roomsTotal=4".format(urllib.parse.quote(max_rent_apartment.unified_address.iloc[0])))
print("https://realty.yandex.ru/otsenka-kvartiry-po-adresu-onlayn/{}/snyat/kvartira/".format(urllib.parse.quote(max_rent_apartment.unified_address.iloc[0])))
# use filters there to select only apartments with 4 rooms

https://realty.yandex.ru/otsenka-kvartiry-po-adresu-onlayn/%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D1%8F%2C%20%D0%A1%D0%B0%D0%BD%D0%BA%D1%82-%D0%9F%D0%B5%D1%82%D0%B5%D1%80%D0%B1%D1%83%D1%80%D0%B3%2C%20%D0%9F%D0%B0%D1%80%D0%B0%D0%B4%D0%BD%D0%B0%D1%8F%20%D1%83%D0%BB%D0%B8%D1%86%D0%B0%2C%203%D0%BA1/snyat/kvartira/


### Find houses with the most cheapest and the most expensive rent price

In [44]:
# Create a new dataframe with houses and median price per sq m
# This time it should with the data calculated without outliers
house_rent_df_cleaned = rent_df_cleaned.groupby('unified_address').price_per_sq_m.median().reset_index()
house_rent_df_cleaned.rename(columns = {'price_per_sq_m': 'house_price_sqm_median_cleaned'}, inplace = True)

In [45]:
most_expensive_house = house_rent_df_cleaned[house_rent_df_cleaned.house_price_sqm_median_cleaned == house_rent_df_cleaned.house_price_sqm_median_cleaned.max()]
most_expensive_house

,unified_address,house_price_sqm_median_cleaned
16851,"Россия, Санкт-Петербург, проспект Динамо, 6",3705.357143


In [46]:
# you can look at the most expensive house here
# https://realty.yandex.ru/archive/Россия%2C%20Санкт-Петербург%2C%20проспект%20Динамо%2C%206/snyat/kvartira/
print("https://realty.yandex.ru/archive/{}/snyat/kvartira/".format(urllib.parse.quote(most_expensive_house.unified_address.iloc[0])))

https://realty.yandex.ru/archive/%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D1%8F%2C%20%D0%A1%D0%B0%D0%BD%D0%BA%D1%82-%D0%9F%D0%B5%D1%82%D0%B5%D1%80%D0%B1%D1%83%D1%80%D0%B3%2C%20%D0%BF%D1%80%D0%BE%D1%81%D0%BF%D0%B5%D0%BA%D1%82%20%D0%94%D0%B8%D0%BD%D0%B0%D0%BC%D0%BE%2C%206/snyat/kvartira/


#### Find the most cheapest house for rent

In [47]:
most_cheapest_house = house_rent_df_cleaned[house_rent_df_cleaned.house_price_sqm_median_cleaned == house_rent_df_cleaned.house_price_sqm_median_cleaned.min()]
most_cheapest_house

,unified_address,house_price_sqm_median_cleaned
2421,"Россия, Ленинградская область, Кириши, улица Мира, 14",50.0


In [48]:
# we see that it's located in Leningrad Oblast, let's look only in St. Petersburg data
house_rent_df_cleaned_spb = house_rent_df_cleaned[house_rent_df_cleaned.unified_address.str.contains('Россия, Санкт-Петербург')]
house_rent_df_cleaned_spb[house_rent_df_cleaned_spb.house_price_sqm_median_cleaned == house_rent_df_cleaned_spb.house_price_sqm_median_cleaned.min()]

,unified_address,house_price_sqm_median_cleaned
4852,"Россия, Санкт-Петербург, Большая Зеленина улица, 12",76.923077


#### Clean cheap outliers
We see that we should clean the data, the cheapest house above is definitely a mistake, and it is an underground station building. You can check it at https://maps.yandex.ru

In [49]:
# consider only data inside the city without oblast
rent_df_cleaned_spb = rent_df_cleaned[rent_df_cleaned.unified_address.str.contains('Россия, Санкт-Петербург')]
median_price_per_sq_m_in_spb = rent_df_cleaned_spb.price_per_sq_m.median()
print("Median price per sq m in St. Petersburg in rent: {}".format(median_price_per_sq_m_in_spb))

Median price per sq m in St. Petersburg in rent: 555.5555555555555


In [50]:
# look at cheap apartments which are more than two times cheaper 
# than the median value of offers we calculated above (555)
rent_df_cleaned_spb[rent_df_cleaned_spb.price_per_sq_m < 250].sample(10)

,offer_id,first_day_exposition,last_day_exposition,last_price,floor,open_plan,rooms,studio,area,kitchen_area,living_area,agent_fee,renovation,offer_type,category_type,unified_address,building_id,price_per_sq_m,house_price_sqm_median
52128,3614067850649759744,2017-01-30T00:00:00+03:00,2017-05-03T00:00:00+03:00,13000.0,2,False,3,False,55.0,9.0,36.0,0.0,3.0,2,2,"Россия, Санкт-Петербург, улица Нахимова, 6",4746820725978884138,236.363636,573.803036
111978,2939523434608526081,2018-03-06T00:00:00+03:00,2018-03-13T00:00:00+03:00,15000.0,6,False,3,False,87.0,NaN,NaN,NaN,3.0,2,2,"Россия, Санкт-Петербург, Окраинная улица, 9В",4510539508387660402,172.413793,241.715116
162148,2067463740344413441,2017-12-23T00:00:00+03:00,2018-08-03T00:00:00+03:00,15000.0,3,False,2,False,62.0,12.0,NaN,75.0,NaN,2,2,"Россия, Санкт-Петербург, Пушкинский район, посёлок Шушары, территория Славянка, Колпинское шоссе, 38к1",7936294240150895121,241.935484,276.923077
122080,374796921134044161,2018-04-09T00:00:00+03:00,2018-04-25T00:00:00+03:00,17000.0,2,False,3,False,78.0,NaN,NaN,50.0,NaN,2,2,"Россия, Санкт-Петербург, Пушкинский район, посёлок Шушары, территория Славянка, Ростовская улица, 14-16",4153179922410625475,217.948718,264.912281
151307,387279007960986625,2018-06-30T00:00:00+03:00,2018-07-07T00:00:00+03:00,20000.0,14,False,2,False,81.0,14.0,45.0,50.0,NaN,2,2,"Россия, Санкт-Петербург, Бестужевская улица, 7к2",7515042099311896468,246.913580,525.657895
170176,4842857799867085678,2018-08-18T00:00:00+03:00,2018-08-20T00:00:00+03:00,12000.0,7,False,4,False,183.0,30.0,13.0,NaN,NaN,2,2,"Россия, Санкт-Петербург, улица Беринга, 32к2",3768803970881688323,65.573770,282.786885
134433,729466726354615297,2018-05-18T00:00:00+03:00,2018-08-20T00:00:00+03:00,18000.0,4,False,3,False,84.0,8.5,47.0,80.0,3.0,2,2,"Россия, Санкт-Петербург, Пушкинский район, посёлок Шушары, территория Славянка, Колпинское шоссе, 12к1",7936294240150960469,214.285714,405.405405
135124,5565310413834324492,2018-05-21T00:00:00+03:00,2018-05-28T00:00:00+03:00,30400.0,5,False,0,True,130.0,NaN,NaN,100.0,NaN,2,2,"Россия, Санкт-Петербург, Колпино, Тверская улица, 11",1869377462242491750,233.846154,451.709402
151090,1067830251055176960,2018-06-29T00:00:00+03:00,2018-07-17T00:00:00+03:00,18000.0,4,False,2,False,78.6,14.0,48.0,NaN,3.0,2,2,"Россия, Санкт-Петербург, Кронштадт, улица Мануильского, 39",8313552936840434444,229.007634,229.007634
96635,3098921250213083904,2017-12-23T00:00:00+03:00,2017-12-25T00:00:00+03:00,15000.0,5,False,2,False,65.0,NaN,NaN,NaN,11.0,2,2,"Россия, Санкт-Петербург, Пушкинский район, посёлок Шушары, территория Славянка, Изборская улица, 3к1",7727526934365677155,230.769231,310.226493


In [51]:
# it seems that we should clean all apartments where median price is at least 2x higher and cheap 
outliers = rent_df_cleaned_spb[(rent_df_cleaned_spb.price_per_sq_m < 250) 
                               & (rent_df_cleaned_spb.house_price_sqm_median/rent_df_cleaned_spb.price_per_sq_m >= 2)]

In [52]:
print("number of cheap outliers 2x cheaper: {}".format(len(outliers)))

number of cheap outliers 2x cheaper: 100


In [53]:
# clean outliers
rent_df_cleaned_spb = rent_df_cleaned_spb[~((rent_df_cleaned_spb.price_per_sq_m < 250) 
                               & (rent_df_cleaned_spb.house_price_sqm_median/rent_df_cleaned_spb.price_per_sq_m >= 2))]

In [54]:
# look at cheap apartments which were not cleaned with the previous heuristics
rent_df_cleaned_spb[rent_df_cleaned_spb.price_per_sq_m < 200]

,offer_id,first_day_exposition,last_day_exposition,last_price,floor,open_plan,rooms,studio,area,kitchen_area,living_area,agent_fee,renovation,offer_type,category_type,unified_address,building_id,price_per_sq_m,house_price_sqm_median
3924,4857633,2015-12-13T00:00:00+03:00,2016-06-05T00:00:00+03:00,12000.0,6,False,3,False,61.0,7.0,42.0,80.0,3.0,2,2,"Россия, Санкт-Петербург, Красное Село, Стрельнинское шоссе, 6к3",1523555400005784785,196.721311,256.578947
4449,4975278,2015-12-27T00:00:00+03:00,2016-01-12T00:00:00+03:00,13000.0,4,False,3,False,78.0,10.0,47.0,NaN,10.0,2,2,"Россия, Санкт-Петербург, 12-я Красноармейская улица, 7",6204918944542168630,166.666667,166.666667
10428,249021112485953840,2016-03-21T00:00:00+03:00,2016-07-05T00:00:00+03:00,12000.0,3,False,4,False,112.0,14.0,16.0,NaN,0.0,2,2,"Россия, Санкт-Петербург, Свеаборгская улица, 23",4943588578198944211,107.142857,107.142857
14714,8176284694677069056,2016-05-18T00:00:00+03:00,2016-05-18T00:00:00+03:00,14000.0,4,False,3,False,74.0,10.0,NaN,50.0,3.0,2,2,"Россия, Санкт-Петербург, Пушкинский район, посёлок Шушары, квартал Славянка, Ростовская улица, 6к3",4976238169808254780,189.189189,291.963016
15870,5379909360861020417,2016-06-02T00:00:00+03:00,2016-06-09T00:00:00+03:00,14000.0,3,False,3,False,80.0,12.0,NaN,80.0,0.0,2,2,"Россия, Санкт-Петербург, Пушкинский район, посёлок Шушары, квартал Славянка, Ростовская улица, 27",5194982532050895016,175.000000,233.333333
17332,801981697508642560,2016-06-20T00:00:00+03:00,2016-06-29T00:00:00+03:00,12000.0,6,False,5,False,156.0,23.0,16.0,50.0,11.0,2,2,"Россия, Санкт-Петербург, Большая Зеленина улица, 12",7137635118733633172,76.923077,76.923077
17690,6302976505020612958,2016-06-25T00:00:00+03:00,2016-11-08T00:00:00+03:00,17000.0,4,False,3,False,100.0,16.0,20.0,50.0,0.0,2,2,"Россия, Санкт-Петербург, Центральный район, улица Некрасова, 38",4677861887331789522,170.000000,170.000000
18059,3653332947260070401,2016-07-01T00:00:00+03:00,2016-07-08T00:00:00+03:00,11000.0,3,False,2,False,71.0,9.0,42.0,50.0,3.0,2,2,"Россия, Санкт-Петербург, Красное Село, Нагорная улица, 37",4011485438398100794,154.929577,183.098592
25294,6437666954565668864,2016-10-11T00:00:00+03:00,2016-10-13T00:00:00+03:00,15000.0,3,False,2,False,140.0,16.0,19.0,50.0,0.0,2,2,"Россия, Санкт-Петербург, Витебская улица, 4",5256601417414077002,107.142857,107.142857
28181,7660502354047186177,2016-11-18T00:00:00+03:00,2017-01-05T00:00:00+03:00,16000.0,3,False,2,False,82.0,18.0,35.0,50.0,3.0,2,2,"Россия, Санкт-Петербург, Петергоф, Ропшинское шоссе, 3к2",5316955799036440746,195.121951,270.971380


In [55]:
# there are cases where houses had just one offer and house median equals offer's price per square meter
# let's remove these outliers
rent_df_cleaned_spb = rent_df_cleaned_spb[~((rent_df_cleaned_spb.price_per_sq_m < 200) 
                                          & (rent_df_cleaned_spb.price_per_sq_m == rent_df_cleaned_spb.house_price_sqm_median))]

In [56]:
# look at the cheapest apartment
cheapest_offer = rent_df_cleaned_spb[rent_df_cleaned_spb.price_per_sq_m == rent_df_cleaned_spb.price_per_sq_m.min()]
cheapest_offer

,offer_id,first_day_exposition,last_day_exposition,last_price,floor,open_plan,rooms,studio,area,kitchen_area,living_area,agent_fee,renovation,offer_type,category_type,unified_address,building_id,price_per_sq_m,house_price_sqm_median
18059,3653332947260070401,2016-07-01T00:00:00+03:00,2016-07-08T00:00:00+03:00,11000.0,3,False,2,False,71.0,9.0,42.0,50.0,3.0,2,2,"Россия, Санкт-Петербург, Красное Село, Нагорная улица, 37",4011485438398100794,154.929577,183.098592


#### Recalculate houses price medians on the cleaned data and find the cheapest house

In [57]:
house_rent_df_cleaned_spb = rent_df_cleaned_spb.groupby('unified_address').price_per_sq_m.median().reset_index()
house_rent_df_cleaned_spb.rename(columns = {'price_per_sq_m': 'house_price_sqm_median_cleaned'}, inplace = True)
cheapest_house = house_rent_df_cleaned_spb[house_rent_df_cleaned_spb.house_price_sqm_median_cleaned == house_rent_df_cleaned_spb.house_price_sqm_median_cleaned.min()]
cheapest_house

,unified_address,house_price_sqm_median_cleaned
5245,"Россия, Санкт-Петербург, Красное Село, Нагорная улица, 37",183.098592


In [58]:
# looking at the map we understand why it's so cheap: it's far away from the center of the city
# https://realty.yandex.ru/archive/Россия%2C%20Санкт-Петербург%2C%20Красное%20Село%2C%20Нагорная%20улица%2C%2037/snyat/kvartira/
print("https://realty.yandex.ru/archive/{}/snyat/kvartira/".format(urllib.parse.quote(cheapest_house.unified_address.iloc[0])))

https://realty.yandex.ru/archive/%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D1%8F%2C%20%D0%A1%D0%B0%D0%BD%D0%BA%D1%82-%D0%9F%D0%B5%D1%82%D0%B5%D1%80%D0%B1%D1%83%D1%80%D0%B3%2C%20%D0%9A%D1%80%D0%B0%D1%81%D0%BD%D0%BE%D0%B5%20%D0%A1%D0%B5%D0%BB%D0%BE%2C%20%D0%9D%D0%B0%D0%B3%D0%BE%D1%80%D0%BD%D0%B0%D1%8F%20%D1%83%D0%BB%D0%B8%D1%86%D0%B0%2C%2037/snyat/kvartira/


### Analyze how many offers have the commission 

In [59]:
# Some offers has commision fee for the agent (agent_fee)
# With 'value_counts' function find out the share of corresponding value
rent_df_cleaned_spb.agent_fee.value_counts(normalize=True, dropna=False)

agent_fee
100.000000    0.328129
50.000000     0.293380
NaN           0.209340
0.000000      0.035378
90.000000     0.034537
                ...   
9.000000      0.000006
27.000000     0.000006
64.997665     0.000006
11.000000     0.000006
36.000000     0.000006
Name: proportion, Length: 102, dtype: float64

We see that the most popular commission is 100% - 33% of offers. 21% of offers don't have any information about commission.

### Self-control stops
1. Calculate median and mean prices for apartments for rent after cleaning the data in St.Petersburg without Leningrad Oblast. Which of the statistics changed more and why?
2. Calculate median and mean prices for apartments for sell before cleaning the data.
2. Find houses with the most cheapest and most expensive price per sq m in St. Petersburg without Leningrad Oblast after cleaning outliers.
3. Find the most expensive and the most cheapest apartment in St. Petersburg after cleaning outliers.
4. Calculate how many years does it take to cover all money spent on buying apartment by renting it. Find houses in which it's the most effective to invest in buying if you plan to rent and houses in which it will be the most ineffective.

<div class="alert alert-block alert-info">
Task 1. Calculate median and mean prices for apartments for rent after cleaning the data in St.Petersburg without Leningrad Oblast. Which of the statistics changed more and why?
</div>

In [60]:
rent_median_price = rent_df_cleaned_spb.last_price.median()
rent_mean_price = rent_df_cleaned_spb.last_price.mean()
print("Rent median price: {}".format(rent_median_price))
print("Rent mean price: {}".format(rent_mean_price))

Rent median price: 25000.0
Rent mean price: 32605.087189685033


<div class="alert alert-block alert-info">
Task 2. Calculate median and mean prices for apartments for sell before cleaning the data.
</div>

In [61]:
sell_median_price = sell_df.last_price.median()
sell_mean_price = sell_df.last_price.mean()
print("Sell median price: {}".format(sell_median_price))
print("Sell mean price: {}".format(sell_mean_price))

Sell median price: 4599000.0
Sell mean price: 6969288.146034318


<div class="alert alert-block alert-info">
Task 3. Find houses with the most cheapest and most expensive price per sq m in St. Petersburg without Leningrad Oblast after cleaning outliers.
</div>

In [62]:
# Before cleaning
sell_max_price = sell_df.last_price.max()
sell_min_price = sell_df.last_price.min()
sell_median_price = sell_df.last_price.median()
sell_mean_price = sell_df.last_price.mean()


print("Sell max price: {}".format(sell_max_price))
print("Sell min price: {}".format(sell_min_price))
print("Sell median price: {}".format(sell_median_price))
print("Sell mean price: {}".format(sell_mean_price))

Sell max price: 763000000.0
Sell min price: 19000.0
Sell median price: 4599000.0
Sell mean price: 6969288.146034318


In [63]:
sell_df_max = sell_df[sell_df.last_price == sell_max_price]
sell_df_max

,offer_id,first_day_exposition,last_day_exposition,last_price,floor,open_plan,rooms,studio,area,kitchen_area,living_area,agent_fee,renovation,offer_type,category_type,unified_address,building_id
224434,619165944711730944,2017-09-30T00:00:00+03:00,2017-11-02T00:00:00+03:00,763000000.0,10,False,7,False,400.0,NaN,250.0,NaN,1.0,1,2,"Россия, Санкт-Петербург, Мичуринская улица, 4",7962972499404580673


In [64]:
sell_df_min = sell_df[sell_df.last_price == sell_min_price]
sell_df_min

,offer_id,first_day_exposition,last_day_exposition,last_price,floor,open_plan,rooms,studio,area,kitchen_area,living_area,agent_fee,renovation,offer_type,category_type,unified_address,building_id
70001,3438368921946229505,2016-06-16T00:00:00+03:00,2016-06-19T00:00:00+03:00,19000.0,4,False,1,True,39.0,NaN,18.0,NaN,11.0,1,2,"Россия, Санкт-Петербург, улица Ленсовета, 69к1",8047169421659571687


In [65]:
# Create a new column price_per_sq_m and calculate price per sq m
sell_df['price_per_sq_m'] = sell_df.last_price/sell_df.area

/tmp/ipykernel_6846/3164263733.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sell_df['price_per_sq_m'] = sell_df.last_price/sell_df.area


In [66]:
sell_df.head(1)

,offer_id,first_day_exposition,last_day_exposition,last_price,floor,open_plan,rooms,studio,area,kitchen_area,living_area,agent_fee,renovation,offer_type,category_type,unified_address,building_id,price_per_sq_m
0,5490504238877583424,2014-11-27T00:00:00+03:00,2016-03-14T00:00:00+03:00,3676740.0,19,False,1,False,27.4,16.56,16.56,NaN,0.0,1,2,"Россия, Санкт-Петербург, улица Шкапина, 9-11",3552413344225333068,134187.591241


In [67]:
# find what's median and mean price per square meter in sell
median_price_per_sq_m = sell_df.price_per_sq_m.median()
mean_price_per_sq_m = sell_df.price_per_sq_m.mean()

print("Median price per sq m in rent: {}".format(median_price_per_sq_m))
print("Mean price per sq m in rent: {}".format(mean_price_per_sq_m))

Median price per sq m in rent: 95214.28571428571
Mean price per sq m in rent: 102713.88007995683


#### Find median price per sq m per house

In [68]:
# .groupby function groups dataframe by a selected column (unified_address)
# For each address we then take a median of price per sq m and return a new dataframe 
# with a result
house_sell_df = sell_df.groupby('unified_address').price_per_sq_m.median().reset_index()
house_sell_df.head(3)

,unified_address,price_per_sq_m
0,"Россия, Ленинградская область, Бокситогорск, Городская улица, 1",15316.249610
1,"Россия, Ленинградская область, Бокситогорск, Городская улица, 3",19080.327869
2,"Россия, Ленинградская область, Бокситогорск, Городская улица, 4",20812.534664


In [69]:
# Rename the column with a 'rename' function in this new dataframe
house_sell_df.rename(columns = {'price_per_sq_m': 'house_price_sqm_median'}, inplace = True)

#### Merge sell data with house median prices and inspect outliers

In [70]:
# With a 'merge' function join two dataframes.
# As these 2 dataframes have only one common column name (unified_address) it will merge dataframes by it
sell_df = sell_df.merge(house_sell_df)

In [71]:
# Assign outliers if price per sq m is 5 times higher than the median value in the house
outliers = sell_df[(sell_df.price_per_sq_m/sell_df.house_price_sqm_median) > 5]
print(len(outliers))

52


In [72]:
outliers.sample(5)

,offer_id,first_day_exposition,last_day_exposition,last_price,floor,open_plan,rooms,studio,area,kitchen_area,living_area,agent_fee,renovation,offer_type,category_type,unified_address,building_id,price_per_sq_m,house_price_sqm_median
104745,693837914213651457,2017-03-31T00:00:00+03:00,2017-06-03T00:00:00+03:00,45000000.0,5,False,2,False,47.20,8.00,31.00,NaN,3.0,1,2,"Россия, Санкт-Петербург, Белградская улица, 24",1872763988793782544,9.533898e+05,100000.000000
1036,5028522700590625318,2015-01-21T00:00:00+03:00,2017-01-09T00:00:00+03:00,715000000.0,1,False,9,False,550.00,NaN,NaN,NaN,0.0,1,2,"Россия, Санкт-Петербург, Большая Пушкарская улица, 1",2009094821262044658,1.300000e+06,130000.000000
6463,3255552,2015-07-14T00:00:00+03:00,2016-02-20T00:00:00+03:00,13338780.0,2,False,1,False,25.26,5.12,14.14,NaN,2.0,1,2,"Россия, Ленинградская область, Всеволожский район, Сертолово, Пограничная улица, 3к1",2155488592838538573,5.280594e+05,53000.000000
49756,8457217352584272433,2016-05-24T00:00:00+03:00,2017-02-27T00:00:00+03:00,151000000.0,1,False,5,False,120.00,NaN,NaN,NaN,0.0,1,2,"Россия, Санкт-Петербург, улица Профессора Попова, 2",6439896804325032024,1.258333e+06,141780.388908
43024,9051496343777975040,2016-04-14T00:00:00+03:00,2016-07-25T00:00:00+03:00,85501560.0,2,False,4,False,190.00,21.00,133.00,NaN,1.0,1,2,"Россия, Санкт-Петербург, Литейный проспект, 24",7528309176946493978,4.500082e+05,81411.126187


#### Clean data from the outliers

In [73]:
# Let's create a new dataframe with clean data.
# With ~ we can select the data which does NOT correspond to the required conditions
sell_df_cleaned = sell_df[~((sell_df.price_per_sq_m/sell_df.house_price_sqm_median) > 5)]

In [74]:
sell_df_cleaned = sell_df_cleaned[(sell_df_cleaned.last_price >= 1000000)]

In [75]:
# we can see that outliers are those which are in houses with price per sq m below 1000 rubles, 
# and those which are the same with median, in which case it's the only offer per house
outliers_count = len(sell_df_cleaned[(sell_df_cleaned.price_per_sq_m > 500000) 
                                     & ((sell_df_cleaned.house_price_sqm_median < 200000) 
                                        | (sell_df_cleaned.house_price_sqm_median == sell_df_cleaned.price_per_sq_m))])
print("outliers found: {}".format(outliers_count))
sell_df_cleaned = sell_df_cleaned[~((sell_df_cleaned.price_per_sq_m > 500000) 
                                     & ((sell_df_cleaned.house_price_sqm_median < 200000) 
                                        | (sell_df_cleaned.house_price_sqm_median == sell_df_cleaned.price_per_sq_m)))]

outliers found: 49


In [76]:
# now max price should be correct, lets look at this offer
max_sell_apartment = sell_df_cleaned[sell_df_cleaned.last_price == sell_df_cleaned.last_price.max()]
max_sell_apartment

,offer_id,first_day_exposition,last_day_exposition,last_price,floor,open_plan,rooms,studio,area,kitchen_area,living_area,agent_fee,renovation,offer_type,category_type,unified_address,building_id,price_per_sq_m,house_price_sqm_median
199146,8310251211233707984,2018-03-26T00:00:00+03:00,2018-08-20T00:00:00+03:00,605740032.0,2,False,5,False,443.0,10.0,248.0,0.0,NaN,1,2,"Россия, Санкт-Петербург, Южная дорога, 5",3312559599398519904,1.367359e+06,759301.442673


In [77]:
# let's look at house address. 
# with 'iloc' property we can take any row by index, 
# in this case we take the first item to get the address
max_sell_apartment.unified_address.iloc[0]

'Россия, Санкт-Петербург, Южная дорога, 5'

### Find houses with the most cheapest and the most expensive sell price

In [78]:
# Create a new dataframe with houses and median price per sq m
# This time it should with the data calculated without outliers
house_sell_df_cleaned = sell_df_cleaned.groupby('unified_address').price_per_sq_m.median().reset_index()
house_sell_df_cleaned.rename(columns = {'price_per_sq_m': 'house_price_sqm_median_cleaned'}, inplace = True)

In [79]:
most_expensive_house = house_sell_df_cleaned[house_sell_df_cleaned.house_price_sqm_median_cleaned == house_sell_df_cleaned.house_price_sqm_median_cleaned.max()]
most_expensive_house

,unified_address,house_price_sqm_median_cleaned
22334,"Россия, Санкт-Петербург, Санаторная аллея, 3",818152.92298


#### Find the most cheapest house for sell

In [80]:
most_cheapest_house = house_sell_df_cleaned[house_sell_df_cleaned.house_price_sqm_median_cleaned == house_sell_df_cleaned.house_price_sqm_median_cleaned.min()]
most_cheapest_house

,unified_address,house_price_sqm_median_cleaned
15560,"Россия, Санкт-Петербург, Конторская улица, 16",5000.0


In [81]:
# we see that it's located in Leningrad Oblast, let's look only in St. Petersburg data
house_sell_df_cleaned_spb = house_sell_df_cleaned[house_sell_df_cleaned.unified_address.str.contains('Россия, Санкт-Петербург')]
house_sell_df_cleaned_spb[house_sell_df_cleaned_spb.house_price_sqm_median_cleaned == house_sell_df_cleaned_spb.house_price_sqm_median_cleaned.min()]

,unified_address,house_price_sqm_median_cleaned
15560,"Россия, Санкт-Петербург, Конторская улица, 16",5000.0


#### Clean cheap outliers
We see that we should clean the data, the cheapest house above is definitely a mistake, and it is an underground station building. You can check it at https://maps.yandex.ru

In [82]:
# consider only data inside the city without oblast
sell_df_cleaned_spb = sell_df_cleaned[sell_df_cleaned.unified_address.str.contains('Россия, Санкт-Петербург')]
median_price_per_sq_m_in_spb = sell_df_cleaned_spb.price_per_sq_m.median()
print("Median price per sq m in St. Petersburg in rent: {}".format(median_price_per_sq_m_in_spb))

Median price per sq m in St. Petersburg in rent: 102272.72727272728


In [83]:
# look at cheap apartments which are more than two times cheaper 
# than the median value of offers we calculated above
sell_df_cleaned_spb[sell_df_cleaned_spb.price_per_sq_m < (median_price_per_sq_m_in_spb/2)].sample(3)

,offer_id,first_day_exposition,last_day_exposition,last_price,floor,open_plan,rooms,studio,area,kitchen_area,living_area,agent_fee,renovation,offer_type,category_type,unified_address,building_id,price_per_sq_m,house_price_sqm_median
205975,8290493366304563457,2018-04-09T00:00:00+03:00,2018-05-24T00:00:00+03:00,3500000.0,2,False,2,False,70.0,NaN,NaN,NaN,NaN,1,2,"Россия, Санкт-Петербург, Колпинский район, посёлок Понтонный, Заводская улица, 18",6103388847815428933,50000.000000,55537.728027
64437,6375826089739418625,2016-09-02T00:00:00+03:00,2016-10-17T00:00:00+03:00,1750000.0,4,False,0,True,44.6,NaN,16.5,NaN,0.0,1,2,"Россия, Санкт-Петербург, улица Костюшко, 13к1",6774259227762089930,39237.668161,99888.888889
56676,7278886061192029441,2016-07-12T00:00:00+03:00,2016-07-18T00:00:00+03:00,10400000.0,6,False,2,False,765.0,NaN,40.5,NaN,10.0,1,2,"Россия, Санкт-Петербург, Яхтенная улица, 3к2",789694191900325461,13594.771242,144531.250000


In [84]:
# it seems that we should clean all apartments where median price is at least 2x higher and cheap 
outliers = sell_df_cleaned_spb[(sell_df_cleaned_spb.price_per_sq_m < (median_price_per_sq_m_in_spb/2)) 
                               & (sell_df_cleaned_spb.house_price_sqm_median/sell_df_cleaned_spb.price_per_sq_m >= 2)]

In [85]:
print("number of cheap outliers 2x cheaper: {}".format(len(outliers)))

number of cheap outliers 2x cheaper: 304


In [86]:
# clean outliers
sell_df_cleaned_spb = sell_df_cleaned_spb[~((sell_df_cleaned_spb.price_per_sq_m < (median_price_per_sq_m_in_spb/2)) 
                               & (sell_df_cleaned_spb.house_price_sqm_median/sell_df_cleaned_spb.price_per_sq_m >= 2))]

In [87]:
# look at cheap apartments which were not cleaned with the previous heuristics
sell_df_cleaned_spb[sell_df_cleaned_spb.price_per_sq_m < ((median_price_per_sq_m_in_spb/2))]

,offer_id,first_day_exposition,last_day_exposition,last_price,floor,open_plan,rooms,studio,area,kitchen_area,living_area,agent_fee,renovation,offer_type,category_type,unified_address,building_id,price_per_sq_m,house_price_sqm_median
351,9069199666252787288,2014-12-10T00:00:00+03:00,2016-09-03T00:00:00+03:00,6390000.0,3,False,3,False,125.00,14.3,67.3,NaN,0.0,1,2,"Россия, Санкт-Петербург, Ломоносов, Еленинская улица, 24",7877007895598460542,51120.000000,53581.871345
1156,1131508,2015-01-28T00:00:00+03:00,2016-10-30T00:00:00+03:00,4300000.0,1,False,3,False,94.00,10.0,58.0,NaN,2.0,1,2,"Россия, Санкт-Петербург, Колпино, Заводской проспект, 62",1549642848657229180,45744.680851,69380.289380
1376,9069199666259219069,2015-02-10T00:00:00+03:00,2016-05-05T00:00:00+03:00,3650000.0,2,False,4,False,98.00,12.0,69.0,NaN,0.0,1,2,"Россия, Санкт-Петербург, Сестрорецк, Приморское шоссе, 250",6439130807317095933,37244.897959,37244.897959
1893,1467881,2015-02-19T00:00:00+03:00,2017-08-26T00:00:00+03:00,3400000.0,11,False,3,False,86.45,12.5,47.4,NaN,7.0,1,2,"Россия, Санкт-Петербург, Пушкинский район, посёлок Шушары, Валдайская улица, 9",9172002365895310872,39329.091961,56118.808810
2377,7110103192797273846,2015-03-11T00:00:00+03:00,2016-10-12T00:00:00+03:00,4100000.0,8,False,3,False,82.50,14.0,50.0,NaN,0.0,1,2,"Россия, Санкт-Петербург, Пушкинский район, посёлок Шушары, Первомайская улица, 17",5131909042114806906,49696.969697,61526.340996
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
254051,8056737725713679872,2018-08-12T00:00:00+03:00,2018-08-20T00:00:00+03:00,2600000.0,1,False,2,False,52.80,6.7,32.7,NaN,2.0,1,2,"Россия, Санкт-Петербург, посёлок Лисий Нос, Ивановская улица, 12к4",2183408768595820797,49242.424242,49242.424242
255093,5252783036233755905,2018-07-22T00:00:00+03:00,2018-08-20T00:00:00+03:00,4600000.0,19,False,3,False,91.25,11.1,48.7,NaN,2.0,1,2,"Россия, Санкт-Петербург, посёлок Парголово, Тихоокеанская улица, 10к1",1273866048480699886,50410.958904,57668.711656
255492,2554639205173457741,2018-08-15T00:00:00+03:00,2018-08-20T00:00:00+03:00,1300000.0,1,False,1,False,38.50,15.2,11.4,NaN,NaN,1,2,"Россия, Санкт-Петербург, Казначейская улица, 3",2257983809765554698,33766.233766,65699.684930
255652,5177752565079589149,2018-02-01T00:00:00+03:00,2018-08-20T00:00:00+03:00,2050000.0,1,False,3,False,55.00,7.2,37.4,NaN,NaN,1,2,"Россия, Санкт-Петербург, посёлок Левашово, улица Кирова, 2",620571518529338168,37272.727273,37272.727273


In [88]:
# there are cases where houses had just one offer and house median equals offer's price per square meter
# let's remove these outliers
sell_df_cleaned_spb = sell_df_cleaned_spb[~((sell_df_cleaned_spb.price_per_sq_m < (median_price_per_sq_m_in_spb/2)) 
                                          & (sell_df_cleaned_spb.price_per_sq_m == sell_df_cleaned_spb.house_price_sqm_median))]

In [89]:
# The most cheapest price per sq m in St. Petersburg without Leningrad Oblast after cleaning outliers
min = sell_df_cleaned_spb[sell_df_cleaned_spb.house_price_sqm_median == sell_df_cleaned_spb.house_price_sqm_median.min()]
print(f"The most cheapest price per sq m in St. Petersburg is: {min['unified_address'].to_string(index=False)}")

The most cheapest price per sq m in St. Petersburg is: Россия, Санкт-Петербург, улица Стойкости, 36к1


In [90]:
# The most expensive price per sq m in St. Petersburg without Leningrad Oblast after cleaning outliers
max = sell_df_cleaned_spb[sell_df_cleaned_spb.house_price_sqm_median == sell_df_cleaned_spb.house_price_sqm_median.max()].head(1)
print(f"The most expensive price per sq m in St. Petersburg is: {max['unified_address'].to_string(index=False)}")

The most expensive price per sq m in St. Petersburg is: Россия, Санкт-Петербург, Санаторная аллея, 3


<div class="alert alert-block alert-info">
Task 4. Find the most expensive and the most cheapest apartment in St. Petersburg after cleaning outliers.
</div>

In [91]:
# The most expensive apartment in St. Petersburg after cleaning outliers
expensive_apartment = sell_df_cleaned_spb[sell_df_cleaned_spb.last_price == sell_df_cleaned_spb.last_price.max()]
expensive_apartment

,offer_id,first_day_exposition,last_day_exposition,last_price,floor,open_plan,rooms,studio,area,kitchen_area,living_area,agent_fee,renovation,offer_type,category_type,unified_address,building_id,price_per_sq_m,house_price_sqm_median
199146,8310251211233707984,2018-03-26T00:00:00+03:00,2018-08-20T00:00:00+03:00,605740032.0,2,False,5,False,443.0,10.0,248.0,0.0,NaN,1,2,"Россия, Санкт-Петербург, Южная дорога, 5",3312559599398519904,1.367359e+06,759301.442673


In [92]:
# The most cheapest apartment in St. Petersburg after cleaning outliers
cheapest_apartment = sell_df_cleaned_spb[sell_df_cleaned_spb.last_price == sell_df_cleaned_spb.last_price.min()]
cheapest_apartment

,offer_id,first_day_exposition,last_day_exposition,last_price,floor,open_plan,rooms,studio,area,kitchen_area,living_area,agent_fee,renovation,offer_type,category_type,unified_address,building_id,price_per_sq_m,house_price_sqm_median
211414,5365283220161389569,2018-04-24T00:00:00+03:00,2018-04-27T00:00:00+03:00,1100000.0,1,False,0,True,33.0,NaN,NaN,NaN,3.0,1,2,"Россия, Санкт-Петербург, Пушкинский район, посёлок Шушары, Первомайская улица, 5к3",6931516357826986190,33333.333333,62500.0


<div class="alert alert-block alert-info">
Task 5. Calculate how many years does it take to cover all money spent on buying apartment by renting it. Find houses in which it's the most effective to invest in buying if you plan to rent and houses in which it will be the most ineffective.
</div>

In [93]:
# Let's group houses by building_id for sell
sell_colomns = sell_df_cleaned_spb[['building_id', 'unified_address', 'last_price']]
sell_table = sell_colomns.groupby(['building_id', 'unified_address'])['last_price'].mean().astype(int).reset_index()
# Let's group houses by building_id for sell
rent_colomns = rent_df_cleaned_spb[['building_id', 'unified_address', 'last_price']]
rent_colomns['annual_price'] = rent_colomns['last_price'] * 12
rent_table = rent_colomns.groupby(['building_id', 'unified_address'])['annual_price'].mean().astype(int).reset_index()

/tmp/ipykernel_6846/1966575709.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rent_colomns['annual_price'] = rent_colomns['last_price'] * 12


In [94]:
# Let's merge tables
table = pd.merge(sell_table, rent_table, on = ['building_id', 'unified_address'], how = 'inner')
# Rename
table = table.rename(
    columns={'last_price': 'avg_price'}
)
# Let's calculate how many years it will take to recoup the money spent on buying an apartment by renting it out
table['years'] = round(table['avg_price'] / table['annual_price'], 2)
table.head(5)

,building_id,unified_address,avg_price,annual_price,years
0,458573008430269,"Россия, Санкт-Петербург, Биржевая линия, 1",5198333,330000,15.75
1,3603962364551559,"Россия, Санкт-Петербург, Учебный переулок, 8к3",5942000,322000,18.45
2,3603962364551561,"Россия, Санкт-Петербург, Учебный переулок, 8к1",5131250,358000,14.33
3,3603962364553483,"Россия, Санкт-Петербург, Учебный переулок, 6к1",4610800,313000,14.73
4,13212466575073986,"Россия, Санкт-Петербург, Дунайский проспект, 34/16",6662580,404689,16.46


In [95]:
# Top 10 apartments worth investing in
table.sort_values(by = 'years', ascending = True).head(10)

,building_id,unified_address,avg_price,annual_price,years
6937,3967985793187418353,"Россия, Санкт-Петербург, Боровая улица, 94",2230000,1800000,1.24
7938,4570380531975985556,"Россия, Санкт-Петербург, 11-я линия Васильевского острова, 18",1430000,960000,1.49
3399,1791270648516624245,"Россия, Санкт-Петербург, Малодетскосельский проспект, 7",1952000,1080000,1.81
4631,2540069387193465645,"Россия, Санкт-Петербург, Гродненский переулок, 4",1950000,1020000,1.91
3103,1627973416240229632,"Россия, Санкт-Петербург, 7-я Красноармейская улица, 7",4600000,2280000,2.02
832,441654007617554222,"Россия, Санкт-Петербург, улица Воскова, 3",2240000,1014857,2.21
4630,2540069387193465644,"Россия, Санкт-Петербург, Гродненский переулок, 3",1950000,780000,2.50
12469,6941707237994729466,"Россия, Санкт-Петербург, Большая Пушкарская улица, 25",1820000,720000,2.53
1917,815795893180413008,"Россия, Санкт-Петербург, Саблинская улица, 4",2556000,900000,2.84
6590,3673956658633628256,"Россия, Санкт-Петербург, 6-я Красноармейская улица, 6",4100000,1440000,2.85


These apartments will pay for themselves within 1-3 years

In [96]:
# Top 10 apartments that are not worth investing in
table.sort_values(by = 'years', ascending = False).head(10)

,building_id,unified_address,avg_price,annual_price,years
16159,9115222827675808888,"Россия, Санкт-Петербург, Моховая улица, 26",126481200,416000,304.04
1197,536301776913345245,"Россия, Санкт-Петербург, Пушкин, Средняя улица, 32",58466666,300000,194.89
11954,6749677014473038546,"Россия, Санкт-Петербург, Алтайская улица, 4",29900000,180000,166.11
3203,1703651731789727303,"Россия, Санкт-Петербург, Казанская улица, 27",33000000,216000,152.78
6376,3567988545862627405,"Россия, Санкт-Петербург, Загородный проспект, 11",55000000,360000,152.78
6235,3474496114170259484,"Россия, Санкт-Петербург, Ковенский переулок, 22-24",42495000,300000,141.65
6726,3800669348402017458,"Россия, Санкт-Петербург, 7-я Советская улица, 10-12",55410000,420000,131.93
1919,817754433912049435,"Россия, Санкт-Петербург, Юризанская улица, 12",70000000,540000,129.63
6134,3406104650917898481,"Россия, Санкт-Петербург, набережная реки Фонтанки, 24",78333333,605538,129.36
2907,1451998268900987936,"Россия, Санкт-Петербург, Бармалеева улица, 2",85000000,660000,128.79


It will take more than 100 years for these apartments to pay off.